<a href="https://colab.research.google.com/github/Mike030668/Spliter_Text2Movi_Kandinskiy_22/blob/main/notebooks/Constract_steps_cickle_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Mike030668/Spliter_Text2Movi_Kandinskiy_22.git -q

# Construct method to find points of start and end fragments movi

In [2]:
import numpy as np
import torch
from torch import nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAIN_DIR =  "/content/Spliter_Text2Movi_Kandinskiy_22"
import sys
sys.path.append(MAIN_DIR)
print(DEVICE)

cuda


## init rand data

In [3]:
from utills.step_points import ComputeDiffPoints, Shuff_Reshuff

In [226]:
cdp = ComputeDiffPoints()

# make set rand embeddings for tests
d_batch = 15
MAX_LEN_MOVI = 100

# sets images embeds
image_embeds = torch.randn(d_batch+1, 1, 1280)

# text embedings prompt video from Text model
text_hid_state = torch.randn((1, 77, 1280), requires_grad=True)

# unclip embedings prompt video from Prior
unclip_embed = torch.randn((1, 1, 1280), requires_grad=True)

# make set time_labels
labels = np.random.choice(np.arange(1, MAX_LEN_MOVI), d_batch+1, replace=False)

# start and end points to use for start normal and back ways
labels[0] = 0
labels[-1] = MAX_LEN_MOVI-1

print(labels)
cdp.time_labels = labels
c

print(f"Time labels {labels}\n")
print('\nNorm_way')
print(f"Start base point {cdp.s_point}")
print(f"Base normal points {cdp.points}\n\n")
print('\nBack_way')
print(f"Start base back_point {cdp.back_s_point}")
print(f"Base back points {cdp.back_points}\n\n")

[ 0 68 60 86 50  4 99 55 63 10 42 24 20 88 66 99]
Time labels [ 0 68 60 86 50  4 99 55 63 10 42 24 20 88 66 99]


Norm_way
Start base point 0
Base normal points [68 60 86 50  4 99 55 63 10 42 24 20 88 66 99]



Back_way
Start base back_point 0
Base back points [33 11 79 75 57 89 36 44  0 95 49 13 39 31 99]




## compute normal and back points for rotation steps

In [ ]:
print(f"Time labels {labels}")
print('\n\nNorm rotation way')
print(f"Base normal points {cdp.points} for start rotation steps \n")
id_img_emb_s = config_norm["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s}")
id_uclip_emb = config_norm["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb}")
id_img_delta = config_norm["id_img_delta"]
print(f"ID delta rotation points {id_img_delta}")
delta = config_norm["delta"]
print(f"Delta between points {delta} normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points} for start rotation back steps \n")
id_img_emb_s = config_back["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s} in base back points")
id_uclip_emb = config_back["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb} in base back points")
id_img_delta = config_back["id_img_delta"]
print(f"ID delta rotation points {id_img_delta} in base back points")
delta = config_back["delta"]
print(f"Delta between points {delta} back points")


Time labels [ 0 14 20  5 35 82 97 80 13 24 42 23  3 63 90 99]


Norm rotation way
Base normal points [14 20  5 35 82 97 80 13 24 42 23  3 63 90 99] for start rotation steps 

ID image and text start rotation points [2, 7, 11, 0, 2, 7, 11, 11, 0, 1, 2, 7, 8, 10, 11, 0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 2, 11, 0, 1, 2, 7, 10, 11, 0, 1, 2, 3, 7, 8, 10, 11, 0, 1, 2, 7, 11, 0, 1, 2, 3, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID uclip_emb start rotation points [2, 7, 11, 0, 2, 7, 11, 11, 0, 1, 2, 7, 8, 10, 11, 0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 2, 11, 0, 1, 2, 7, 10, 11, 0, 1, 2, 3, 7, 8, 10, 11, 0, 1, 2, 7, 11, 0, 1, 2, 3, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID delta rotation points [0, 0, 0, 1, 1, 1, 1, 2, 3, 3, 3,

## compute normal and back points for diff steps

It will be used for prediction from prediction of model

In [218]:
config_diff_norm, config_diff_back = cdp.getpoints_diftrain()

print(f"Time labels {labels}")
print('\n\nNorm diff way')
print(f"Base normal points {cdp.points}")
print(f"Next normal points {cdp.next_points} for diff steps \n")
id_img_emb_s = config_diff_norm["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base normal points")
id_uclip_emb = config_diff_norm["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next normal points")
id_img_delta = config_diff_norm["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next normal points")
delta = config_diff_norm["delta"]
print(f"Delta between diff points {delta} in base normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points}")
print(f"Next back points {cdp.back_next_points} for diff steps \n")
id_img_emb_s = config_diff_back["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base back points")
id_uclip_emb = config_diff_back["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next back points")
id_img_delta = config_diff_back["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next back points")
delta = config_diff_back["delta"]
print(f"Delta between diff points {delta} back points")


Time labels [ 0 77 38 96 11 53 50 44  9 95 29 49 15  7 26 99]


Norm diff way
Base normal points [77 38 96 11 53 50 44  9 95 29 49 15  7 26 99]
Next normal points [ 78  39  97  12  54  51  45  10  96  30  50  16   8  27 100] for diff steps 

ID image and text start diff points [5] in base normal points
ID uclip_emb start diff points [10] in next normal points
ID delta diff points [2] in next normal points
Delta between diff points [46] in base normal points


Norm rotation way
Base back points [73 92 84 50 70  4 90 55 49 46 88  3 61 22 99]
Next back points [ 74  93  85  51  71   5  91  56  50  47  89   4  62  23 100] for diff steps 

ID image and text start diff points [5] in base back points
ID uclip_emb start diff points [11] in next back points
ID delta diff points [3] in next back points
Delta between diff points [46] back points


# Both ways steps model

In [4]:
from models.R_baseline import R_SpliterSimple
from utills.clearing import flush_memory

model = R_SpliterSimple(max_delta_time = 100,
                        emb_dim = 1280,
                        ways = 2
                        )

## Losses

nn.CosineEmbeddingLoss
https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html

In [5]:
clip_text_last_hidden_state = torch.randn((2, 77, 1280)) #outputs.last_hidden_state
embeddings = torch.randn((2, 1, 1280))
delta = torch.tensor([[3],[5]])
direction  = torch.tensor([[0],[1]])
model.to(DEVICE)
out_embeddings = model(text_hidden_states = clip_text_last_hidden_state.to(DEVICE),
                       prior_embeds = embeddings.to(DEVICE),
                       delta_time = delta.to(DEVICE),
                       direction = direction.to(DEVICE)
                        )
out_embeddings.shape

torch.Size([2, 1, 1280])

In [6]:
# https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html
loss = nn.CosineEmbeddingLoss()
input1 = torch.randn(3, 1, requires_grad=True)
input2 = torch.randn(3, 5, requires_grad=True)
target = torch.ones(3)
output = loss(input1, input2, target)
output.backward()
output

tensor(0.0415, grad_fn=<MeanBackward0>)

## apply as vector losses

In [7]:
mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
cos_loss = nn.CosineEmbeddingLoss(reduction = 'none').to(DEVICE)
target = torch.ones(1280).to(DEVICE)
loss_cos = cos_loss(embeddings.squeeze(1).T.to(DEVICE), out_embeddings.squeeze(1).T, target)
print(loss_cos)

tensor([1.8024, 1.9816, 0.0150,  ..., 0.3603, 0.0881, 1.3852], device='cuda:0',
       grad_fn=<AddBackward0>)


## first steps normal and back shuffleed

In [12]:
all_points = np.append(cdp.points, cdp.back_points)
# intit class for shufflee
b_srs = Shuff_Reshuff(len(all_points))
bs_idx = b_srs.shuffle() # shuffleed indexees
bu_idx = b_srs.unshuffle() # indexees for re_shuffle

B_batch = 2*d_batch

time_base = torch.tensor(all_points).unsqueeze(1)
directions = torch.concat([torch.zeros_like(time_base[:d_batch]), torch.ones_like(time_base[d_batch:])])

# shuffleed
time_base = time_base.to(DEVICE)
directions = directions.to(DEVICE)


text_hid_states =  torch.concat([text_hid_state for _ in range(B_batch)])
base_unclip_embs = torch.concat([unclip_embed for _ in range(B_batch)])

base_img_embs = [image_embeds[0] for _ in range(d_batch)] + [image_embeds[-1] for _ in range(d_batch)]
base_img_embs = torch.concat(base_img_embs)


back_img_embs = torch.concat([image_embeds[i,:,:] for i in range(image_embeds.shape[0])][::-1]).unsqueeze(1)
img_embs =[image_embeds[1:,:,:], back_img_embs[1:,:,:]]
img_embs = torch.concat(img_embs)


pred_unclip_embs = model(
      text_hidden_states = text_hid_states[bs_idx].to(torch.float32).to(DEVICE), # shuffleed
      prior_embeds = base_unclip_embs[bs_idx].to(torch.float32).to(DEVICE),
      delta_time = time_base[bs_idx],
      direction = directions[bs_idx]
                        )

# difference
diff_img_embs =  (base_img_embs[bs_idx].squeeze(dim=1) - img_embs[bs_idx].squeeze(dim=1)) #
diff_unclip_embs = (base_unclip_embs[bs_idx].squeeze(dim=1).to(DEVICE) - pred_unclip_embs.squeeze(dim=1))

# CosineEmbeddingLoss between difference
movi_cos_loss = cos_loss(diff_unclip_embs.T, diff_img_embs.T.to(DEVICE), target).half()  #

# MSELoss between predict and
movi_mse_loss = mse_loss(diff_unclip_embs.half(), diff_img_embs.half().to(DEVICE))#

movi_loss = movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

del(time_base, directions, diff_img_embs, diff_unclip_embs)
flush_memory()

print(movi_loss)

tensor([ 2.1621, 10.9531,  7.4219,  ...,  2.7754,  1.9805,  2.3164],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>)


## Rotation steps normal and back shuffleed

In [232]:
class RotationVectors:
    def unit_vector(self, vectors):
        """
        TORCH
        Returns the unit vectors of the input vectors.
        vectors: torch tensor of shape (batch_size, vector_dim)
        """
        return vectors / torch.norm(vectors, dim=1, keepdim=True)

    def angle(self, vectors1, vectors2):
        """
        Returns the angles in radians between the given batches of vectors.
        vectors1, vectors2: torch tensors of shape (batch_size, vector_dim)
        """
        v1_u = self.unit_vector(vectors1)
        v2_u = self.unit_vector(vectors2)

        minor = torch.det(torch.stack((v1_u[:, -2:], v2_u[:, -2:]), dim=1))
        if (minor == 0).any():
            raise NotImplementedError('Some vectors are too odd!')
        return torch.sign(minor) * torch.acos(torch.clamp(torch.sum(v1_u * v2_u, dim=1), -1.0, 1.0))

    def get_rotation_matrix(self, vec_1, vec_2):
        """
        Compute rotation matrices between batches of vectors.
        vec_1, vec_2: torch tensors of shape (batch_size, vector_dim)
        """
        a = self.angle(vec_1, vec_2)
        n1 = self.unit_vector(vec_1)

        vec_2 = vec_2 - torch.sum(n1 * vec_2, dim=1, keepdim=True) * n1
        n2 = self.unit_vector(vec_2)

        # Assuming vec_1, n1, n2, and a are already defined as torch tensors
        I = torch.eye(vec_1.shape[1]).to(vec_1.device)
        R = I + ((n2.unsqueeze(2) @ n1.unsqueeze(1)) - (n1.unsqueeze(2) @ n2.unsqueeze(1))) * torch.sin(a).unsqueeze(1).unsqueeze(2) + \
            ((n1.unsqueeze(2) @ n1.unsqueeze(1)) + (n2.unsqueeze(2) @ n2.unsqueeze(1))) * (torch.cos(a).unsqueeze(1).unsqueeze(2) - 1)

        return R

    def cosim_rotate(self, b_matrix, cos_sim_matrix, b_R_matrix):
         """
         Rotate batch_vectors as matrix with batch R_matrixes
         with using batch cos_sim these batch_vectors to engine vectors
         """
         return torch.mul((1 - cos_sim_matrix).unsqueeze(1), b_matrix) + torch.mul(cos_sim_matrix.unsqueeze(1), b_matrix @ b_R_matrix)

In [210]:
#from utills.rotations import RotationVectors
# inite rotation class
RV = RotationVectors()

In [215]:
rote_norm_train = len(config_norm['id_uclip_emb'])
rote_back_train = len(config_back['id_uclip_emb'])
rote_train =  rote_norm_train + rote_back_train

if rote_train:
    text_hid_states_2rt = []
    unclip_embs_2rt = []
    base_img_embs_2rt = []
    image_embs_2rt = []
    delta_2rt = []
    direction_2rt = []

    # intit class for shufflee again
    srs = Shuff_Reshuff(rote_train)
    s_idx = srs.shuffle() # shuffleed indexees
    u_idx = srs.unshuffle() # indexees for re_shuffle


    if rote_norm_train:
        # collect norm steps
        text_hid_states_2rt.append(torch.clone(text_hid_states[:d_batch])[config_norm['id_uclip_emb']])
        unclip_embs_2rt.append(torch.clone(base_unclip_embs[:d_batch])[config_norm['id_uclip_emb']])
        base_img_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_emb_s']])
        image_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_delta']])
        delta_2rt.append(torch.tensor(config_norm['delta']).unsqueeze(1))
        direction_2rt.append(torch.zeros_like(delta_2rt[-1]))


    if rote_back_train:
        # collect back steps
        text_hid_states_2rt.append(torch.clone(text_hid_states[d_batch:])[config_back['id_uclip_emb']])
        unclip_embs_2rt.append(torch.clone(base_unclip_embs[d_batch:])[config_back['id_uclip_emb']])
        base_img_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_emb_s']])
        image_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_delta']])
        delta_2rt.append(torch.tensor(config_back['delta']).unsqueeze(1))
        direction_2rt.append(torch.ones_like(delta_2rt[-1]))

    # concat tansors
    if rote_train > B_batch:
       R_batch = B_batch
    else: R_batch = rote_train

    # shufle and take R_batch samples
    text_hid_states_2rt = torch.concat(text_hid_states_2rt)[s_idx][:R_batch].to(DEVICE)
    unclip_embs_2rt = torch.concat(unclip_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    base_img_embs_2rt = torch.concat(base_img_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    image_embs_2rt = torch.concat(image_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    delta_2rt = torch.concat(delta_2rt)[s_idx][s_idx][:R_batch].to(DEVICE)
    direction_2rt = torch.concat(direction_2rt)[s_idx][:R_batch].to(DEVICE)

    # get cos_sim  base_img_embs vectors and base_unclip_embs vectors
    cos_sim = torch.cosine_similarity(base_img_embs.unsqueeze(1), base_unclip_embs, dim = 2)

    # get rotation marixes_i2i
    R_marixes_i2i = RV.get_rotation_matrix(base_img_embs.to(DEVICE), base_img_embs_2rt.squeeze(1)).to(DEVICE)

    # compute roted unclip_embs with R_marixes_i2i and cos_sim base_img_embs and base_unclip_embs
    unclip_embs_2rt = RV.cosim_rotate(unclip_embs_2rt, cos_sim.to(DEVICE), R_marixes_i2i)

    # get rotation marixes_u2u
    R_marixes_u2u = RV.get_rotation_matrix(base_unclip_embs.squeeze(1).to(torch.float32).to(DEVICE),
                                                           unclip_embs_2rt.squeeze(1))

    # compute roted text_hid_states with R_marixes_u2u and cos_sim base_img_embs and base_unclip_embs
    text_hid_states_2rt = text_hid_states_2rt @ R_marixes_u2u

    pred_rote_embs = model(
                  text_hidden_states = text_hid_states_2rt.to(torch.float32), # shuffleed
                  prior_embeds = unclip_embs_2rt.to(torch.float32),
                  delta_time = delta_2rt,
                  direction = direction_2rt
                                    )

    # difference
    diff_unclip_embeds = (unclip_embs_2rt.squeeze(dim=1) - pred_rote_embs.squeeze(dim=1))
    diff_img_embs =  (base_img_embs_2rt.squeeze(dim=1) - image_embs_2rt.squeeze(dim=1))

    # CosineEmbeddingLoss between difference
    movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

    # MSELoss between predict and
    movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

    movi_loss = movi_loss + movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

    del(delta_2rt, direction_2rt, diff_img_embs, diff_unclip_embeds)
    del(pred_rote_embs, text_hid_states_2rt, R_marixes_i2i, R_marixes_u2u)
    del(unclip_embs_2rt,image_embs_2rt)
    flush_memory()


    print(movi_loss)




tensor([15.8672, 36.0000, 31.0781,  ..., 18.8438, 16.2969, 14.0156],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>)


## Next steps normal and back from pred steps shuffleed

In [219]:
diff_norm = len(config_diff_norm['id_uclip_emb'])
diff_back = len(config_diff_back['id_uclip_emb'])
diff_train = diff_norm + diff_back

if diff_train:
    take_text_hid_states = []
    take_base_unclip_embs = []
    next_unclip_embs = []
    next_base_img_embs = []
    next_image_embs = []
    next_delta = []
    next_text_hid_states = []
    next_direction = []

    # un_shuffleed
    pred_unclip_embs = torch.clone(pred_unclip_embs.detach().cpu())[bu_idx]

    # intit class for shufflee again
    srs = Shuff_Reshuff(diff_train)
    s_idx = srs.shuffle() # shuffleed indexees
    u_idx = srs.unshuffle() # indexees for re_shuffle

    if diff_norm:
        # collect next norm steps
        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
        take_text_hid_states.append(torch.clone(text_hid_states[:d_batch])[config_diff_norm['id_uclip_emb']])
        next_unclip_embs.append(torch.clone(pred_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
        next_base_img_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_emb_s']])
        next_image_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_delta']])
        next_delta.append(torch.tensor(config_diff_norm['delta']).unsqueeze(1))
        next_direction.append(torch.zeros_like(next_delta[-1]))

    if diff_back:
        # collect next back steps
        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_back['id_uclip_emb']])
        take_text_hid_states.append(torch.clone(text_hid_states[d_batch:])[config_diff_back['id_uclip_emb']])
        next_unclip_embs.append(torch.clone(pred_unclip_embs[d_batch:])[config_diff_back['id_uclip_emb']])
        next_base_img_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_emb_s']])
        next_image_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_delta']])
        next_delta.append(torch.tensor(config_diff_back['delta']).unsqueeze(1))
        next_direction.append(torch.ones_like(next_delta[-1]))



    take_base_unclip_embs = torch.concat(take_base_unclip_embs).to(DEVICE)
    take_text_hid_states = torch.concat(take_text_hid_states).to(DEVICE)
    next_unclip_embs = torch.concat(next_unclip_embs).to(DEVICE)
    next_base_img_embs = torch.concat(next_base_img_embs)
    next_image_embs = torch.concat(next_image_embs)
    next_delta = torch.concat(next_delta)[s_idx].to(DEVICE)
    next_direction = torch.concat(next_direction)[s_idx].to(DEVICE)

    # get rotation vectors
    R_marixes = RV.get_rotation_matrix(take_base_unclip_embs.squeeze(1), next_unclip_embs.squeeze(1))
    next_text_hid_states = take_text_hid_states @ R_marixes


    next_pred_unclip_embs = model(
        text_hidden_states = next_text_hid_states[s_idx].to(torch.float32),
        prior_embeds = next_unclip_embs[s_idx].to(torch.float32),
        delta_time = next_delta,
        direction = next_direction
        )

    # difference
    diff_unclip_embeds = (next_unclip_embs[s_idx].squeeze(dim=1).to(DEVICE) - next_pred_unclip_embs.squeeze(dim=1))
    diff_img_embs =  (next_base_img_embs[s_idx].squeeze(dim=1) - next_image_embs[s_idx].squeeze(dim=1))

    # CosineEmbeddingLoss between difference
    movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

    # MSELoss between predict and
    movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

    next_movi_loss = movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

    del(diff_img_embs, diff_unclip_embeds)
    del(next_image_embs, next_base_img_embs, next_delta, next_direction)
    del(next_text_hid_states,  next_unclip_embs)
    del(pred_unclip_embs, next_pred_unclip_embs)
    flush_memory()

    print(next_movi_loss)

tensor([0.7217, 2.8125, 2.3320,  ..., 3.9258, 2.1113, 0.9697], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddBackward0>)


##  Cicle

### init rand data

In [233]:
all_data = []
# make set rand embeddings for tests
d_batch = 15
q_ty = 100

for i in range(q_ty):
    data = dict()
    # make set time_labels
    labels = np.random.choice(np.arange(1, 99), d_batch+1, replace=False)
    labels[0] = 0
    labels[-1] = 99
    data['labels'] = labels

    data['image_embeds'] = []
    for i in range(d_batch+1):
      if not i : data['image_embeds'].append(torch.randn(1, 1, 1280))
      else:
        next = data['image_embeds'][-1] + 0.0001*torch.randn(1, 1, 1280)
        data['image_embeds'].append(next)

    data['text_hid_state'] = torch.randn((1, 77, 1280))
    data['unclip_embed'] = torch.randn((1, 1, 1280), requires_grad=True)

    all_data.append(data)


In [234]:
#from utills.rotations import RotationVectors
# inite rotation class
LR_RATE = 1e-03
EPS = 0.95

mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
cos_loss = nn.CosineEmbeddingLoss(reduction = 'none').to(DEVICE)
target = torch.ones(1280).to(DEVICE)


optimizer = torch.optim.AdamW(model.parameters(), lr=LR_RATE)

# https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              cycle_momentum = False,
                                              base_lr=LR_RATE/10,
                                              max_lr=LR_RATE,
                                              step_size_up=5,
                                              mode="triangular2")

add_back_train = True
add_rote_train = True
add_diff_train = True

model.to(DEVICE)
model.train()

# inite rotation class
RV = RotationVectors()

# inite ComputeDiffPoints class
cdp = ComputeDiffPoints(treshold = 0)

### test train

In [237]:
# отрисовка прохождения цикла
from tqdm.notebook import tqdm
from random import shuffle
import time

step = 0
EPOHS = 3

hist = {"loss" : [],
        "loss_cos" : [],
        "loss_mse" : [],
        "acc" : [],
        "base_loss" : [],
        "lr" : []
}

for epoch  in tqdm(range(EPOHS), unit ="EPOHS",
                      desc ="Пробегаемся по всем эпохам"):
    eph_loss = 0
    eph_loss_cos = 0
    eph_loss_mse = 0
    eph_cos_acc = 0
    eph_base_loss = 0
    cur_lr = optimizer.param_groups[0]['lr']
    diff_loss_weight = 0

    shuffle(all_data)
    for movi in all_data:

        optimizer.zero_grad()
        # make set rand embeddings for tests
        image_embeds = torch.concat(movi['image_embeds'])
        text_hid_state = movi['text_hid_state']
        unclip_embed = movi['unclip_embed']
        # make set time_labels
        labels = movi['labels']

        # place labels to class points
        cdp.time_labels = labels
        config_norm, config_back = cdp.getpoints_train()
        all_points = cdp.points

        if add_back_train:
            all_points = np.append(all_points, cdp.back_points)

        # intit class for shufflee
        b_srs = Shuff_Reshuff(len(all_points))
        bs_idx = b_srs.shuffle() # shuffleed indexees
        bu_idx = b_srs.unshuffle() # indexees for re_shuffle

        # collect time directions tensors
        time_base = torch.tensor(all_points).unsqueeze(1)

        directions = [torch.zeros_like(time_base[:d_batch])]
        if add_back_train:
            directions.append(torch.ones_like(time_base[d_batch:]))
        directions = torch.concat(directions)


        # collect text tensors
        text_hid_states = [text_hid_state for _ in range(d_batch)]
        if add_back_train:
            text_hid_states.extend([text_hid_state for _ in range(d_batch)])
        text_hid_states =  torch.concat(text_hid_states)

        # collect unclip tensors
        base_unclip_embs = [unclip_embed for _ in range(d_batch)]
        if add_back_train:
            base_unclip_embs.extend([unclip_embed for _ in range(d_batch)])
        base_unclip_embs = torch.concat(base_unclip_embs)

        # collect base_img_embs tensors
        base_img_embs = [image_embeds[0] for _ in range(d_batch)]
        if add_back_train:
            base_img_embs.extend([image_embeds[-1] for _ in range(d_batch)])
        base_img_embs = torch.concat(base_img_embs)

        # collect img_embs tensors
        img_embs = image_embeds[1:]

        if add_back_train:
            # collect back_img_embs tensors
            back_img_embs = torch.flip(image_embeds, [0,])[1:]
            # collect img_embs together tensors
            img_embs = torch.concat([img_embs, back_img_embs])

        # base predict which can used in next step
        pred_unclip_embs = model(
              text_hidden_states = text_hid_states[bs_idx].to(torch.float32).to(DEVICE), # shuffleed
              prior_embeds = base_unclip_embs[bs_idx].to(torch.float32).to(DEVICE),
              delta_time = time_base[bs_idx].to(DEVICE),
              direction = directions[bs_idx].to(DEVICE)
                                )

        # difference
        diff_img_embs = (base_img_embs[bs_idx].squeeze(dim=1) - img_embs[bs_idx].squeeze(dim=1)) #
        diff_unclip_embs = (base_unclip_embs[bs_idx].squeeze(dim=1).to(DEVICE) - pred_unclip_embs.squeeze(dim=1))

        # CosineEmbeddingLoss between difference
        movi_cos_loss = cos_loss(diff_unclip_embs.T, diff_img_embs.T.to(DEVICE), target).half()  #

        # MSELoss between predict and
        movi_mse_loss = mse_loss(diff_unclip_embs.half(), diff_img_embs.half().to(DEVICE)).mean(0)#

        movi_loss_base = movi_cos_loss + movi_mse_loss # (0) for 1280
        eph_base_loss = movi_loss_base.mean().item()

        # CosineEmbeddingLoss between difference
        cos_acc = cos_loss(pred_unclip_embs.squeeze(1).T,
                           img_embs[bs_idx].squeeze(1).T.to(DEVICE), (-1)*target).half()

        eph_cos_acc+= cos_acc.mean().item()

        del(time_base, directions, diff_img_embs, diff_unclip_embs)

        to_rote, rote_norm, rote_back = 0, 0, 0
        if add_rote_train:
            # Rotation train steps
            rote_norm = len(config_norm['id_uclip_emb'])
            rote_back = len(config_back['id_uclip_emb'])
            to_rote =  rote_norm + rote_back

        if add_rote_train and epoch and to_rote:
            text_hid_states_2rt = []
            unclip_embs_2rt = []
            base_img_embs_2rt = []
            image_embs_2rt = []
            delta_2rt = []
            direction_2rt = []

            # intit class for shufflee again
            srs = Shuff_Reshuff(to_rote)
            s_idx = srs.shuffle() # shuffleed indexees
            u_idx = srs.unshuffle() # indexees for re_shuffle


            if rote_norm:
                # collect norm steps
                text_hid_states_2rt.append(torch.clone(text_hid_states[:d_batch])[config_norm['id_uclip_emb']])
                unclip_embs_2rt.append(torch.clone(base_unclip_embs[:d_batch])[config_norm['id_uclip_emb']])
                base_img_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_emb_s']])
                image_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_delta']])
                delta_2rt.append(torch.tensor(config_norm['delta']).unsqueeze(1))
                direction_2rt.append(torch.zeros_like(delta_2rt[-1]))


            if rote_back and add_back_train:
                # collect back steps
                text_hid_states_2rt.append(torch.clone(text_hid_states[d_batch:])[config_back['id_uclip_emb']])
                unclip_embs_2rt.append(torch.clone(base_unclip_embs[d_batch:])[config_back['id_uclip_emb']])
                base_img_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_emb_s']])
                image_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_delta']])
                delta_2rt.append(torch.tensor(config_back['delta']).unsqueeze(1))
                direction_2rt.append(torch.ones_like(delta_2rt[-1]))

            # concat tansors
            if add_back_train:
              if len(direction_2rt) > 2*d_batch:
                R_batch = 2*d_batch
              else: R_batch = len(direction_2rt)
            else:
              if len(direction_2rt) > d_batch:
                R_batch = d_batch
              else: R_batch = len(direction_2rt)

            # shufle and take R_batch samples
            text_hid_states_2rt = torch.concat(text_hid_states_2rt)[s_idx][:R_batch].to(DEVICE)
            unclip_embs_2rt = torch.concat(unclip_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            base_img_embs_2rt = torch.concat(base_img_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            image_embs_2rt = torch.concat(image_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            delta_2rt = torch.concat(delta_2rt)[s_idx][s_idx][:R_batch].to(DEVICE)
            direction_2rt = torch.concat(direction_2rt)[s_idx][:R_batch].to(DEVICE)

            take_base = base_img_embs_2rt.shape[0]
            # get cos_sim  base_img_embs vectors and base_unclip_embs vectors
            cos_sim = torch.cosine_similarity(base_img_embs[:take_base].unsqueeze(1), base_unclip_embs[:take_base], dim = 2)

            # get rotation marixes_i2i
            R_marixes_i2i = RV.get_rotation_matrix(base_img_embs[:take_base].to(DEVICE), base_img_embs_2rt.squeeze(1)).to(DEVICE)

            # compute roted unclip_embs with R_marixes_i2i and cos_sim base_img_embs and base_unclip_embs
            unclip_embs_2rt = RV.cosim_rotate(unclip_embs_2rt, cos_sim.to(DEVICE), R_marixes_i2i)

            # get rotation marixes_u2u
            R_marixes_u2u = RV.get_rotation_matrix(base_unclip_embs[:take_base].squeeze(1).to(torch.float32).to(DEVICE),
                                                                  unclip_embs_2rt.squeeze(1))

            # compute roted text_hid_states with R_marixes_u2u and cos_sim base_img_embs and base_unclip_embs
            text_hid_states_2rt = text_hid_states_2rt @ R_marixes_u2u


            # rotation predict
            pred_rote_embs = model(
                          text_hidden_states = text_hid_states_2rt.to(torch.float32),
                          prior_embeds = unclip_embs_2rt.to(torch.float32),
                          delta_time = delta_2rt,
                          direction = direction_2rt
                                            )

            # difference
            diff_unclip_embeds = (unclip_embs_2rt.squeeze(dim=1) - pred_rote_embs.squeeze(dim=1))
            diff_img_embs = (base_img_embs_2rt.squeeze(dim=1) - image_embs_2rt.squeeze(dim=1))

            rote_loss_weight =hist["acc"][-1]
            #print("rote_loss_weight", rote_loss_weight)
            # CosineEmbeddingLoss between difference
            movi_cos_loss += rote_loss_weight*cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  #

            # MSELoss between predict and
            movi_mse_loss += rote_loss_weight*mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)).mean(0) #

            #movi_loss = movi_loss + movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

            del(delta_2rt, direction_2rt, diff_img_embs, diff_unclip_embeds)
            del(pred_rote_embs, text_hid_states_2rt, R_marixes_i2i, R_marixes_u2u)
            del(unclip_embs_2rt,image_embs_2rt)

        # Diff train steps
        to_diff = 0
        if add_diff_train:
            config_diff_norm, config_diff_back = cdp.getpoints_diftrain()
            diff_norm = len(config_diff_norm['id_uclip_emb'])
            diff_back = len(config_diff_back['id_uclip_emb'])
            to_diff = diff_norm + diff_back

        if add_diff_train and epoch>1 and to_diff:
            diff_loss_weight = 1 - hist["base_loss"][-1]/hist["base_loss"][0]
            if diff_loss_weight<0: diff_loss_weight = 0

        if add_diff_train and epoch>1 and to_diff and diff_loss_weight:
            take_text_hid_states = []
            take_base_unclip_embs = []
            next_unclip_embs = []
            next_base_img_embs = []
            next_image_embs = []
            next_delta = []
            next_text_hid_states = []
            next_direction = []

            # un_shuffleed
            pred_unclip_embs = torch.clone(pred_unclip_embs.detach().cpu())[bu_idx]

            # intit class for shufflee again
            srs = Shuff_Reshuff(to_diff)
            s_idx = srs.shuffle() # shuffleed indexees
            u_idx = srs.unshuffle() # indexees for re_shuffle

            if diff_norm:
                # collect next norm steps
                take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                take_text_hid_states.append(torch.clone(text_hid_states[:d_batch])[config_diff_norm['id_uclip_emb']])
                next_unclip_embs.append(torch.clone(pred_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                next_base_img_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_emb_s']])
                next_image_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_delta']])
                next_delta.append(torch.tensor(config_diff_norm['delta']).unsqueeze(1))
                next_direction.append(torch.zeros_like(next_delta[-1]))

            if diff_back:
                # collect next back steps
                take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_back['id_uclip_emb']])
                take_text_hid_states.append(torch.clone(text_hid_states[d_batch:])[config_diff_back['id_uclip_emb']])
                next_unclip_embs.append(torch.clone(pred_unclip_embs[d_batch:])[config_diff_back['id_uclip_emb']])
                next_base_img_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_emb_s']])
                next_image_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_delta']])
                next_delta.append(torch.tensor(config_diff_back['delta']).unsqueeze(1))
                next_direction.append(torch.ones_like(next_delta[-1]))



            take_base_unclip_embs = torch.concat(take_base_unclip_embs).to(DEVICE)
            take_text_hid_states = torch.concat(take_text_hid_states).to(DEVICE)
            next_unclip_embs = torch.concat(next_unclip_embs).to(DEVICE)
            next_base_img_embs = torch.concat(next_base_img_embs)
            next_image_embs = torch.concat(next_image_embs)
            next_delta = torch.concat(next_delta)[s_idx].to(DEVICE)
            next_direction = torch.concat(next_direction)[s_idx].to(DEVICE)

            # get rotation vectors
            R_marixes = RV.get_rotation_matrix(take_base_unclip_embs.squeeze(1), next_unclip_embs.squeeze(1))
            next_text_hid_states = take_text_hid_states @ R_marixes

            # dif predict from base predict
            next_pred_unclip_embs = model(
                text_hidden_states = next_text_hid_states[s_idx].to(torch.float32),
                prior_embeds = next_unclip_embs[s_idx].to(torch.float32),
                delta_time = next_delta,
                direction = next_direction
                )

            # difference
            diff_unclip_embeds = (next_unclip_embs[s_idx].squeeze(dim=1).to(DEVICE) - next_pred_unclip_embs.squeeze(dim=1))
            diff_img_embs =  (next_base_img_embs[s_idx].squeeze(dim=1) - next_image_embs[s_idx].squeeze(dim=1))


            # CosineEmbeddingLoss between difference
            movi_cos_loss += diff_loss_weight * cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

            # MSELoss between predict and
            movi_mse_loss += diff_loss_weight * mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)).mean(0) #

            del(diff_img_embs, diff_unclip_embeds)
            del(next_image_embs, next_base_img_embs, next_delta, next_direction)
            del(next_text_hid_states,  next_unclip_embs)
            del(pred_unclip_embs, next_pred_unclip_embs)


    # collect loss
    movi_loss = movi_cos_loss + movi_mse_loss # (0) for 1280
    movi_loss.backward(torch.ones_like(movi_loss))
    eph_loss_mse += movi_mse_loss.mean().item()
    eph_loss_cos += movi_cos_loss.mean().item()
    eph_loss += movi_loss.mean().item()


    optimizer.step()

    hist["lr"].append(cur_lr)
    good_steps = len(all_data)
    eph_cos_acc/=good_steps
    eph_base_loss/=good_steps
    eph_loss/=good_steps
    eph_loss_mse/=good_steps
    eph_loss_cos/=good_steps

    hist["loss"].append(eph_loss)
    hist["loss_mse"].append(eph_loss_mse)
    hist["loss_cos"].append(eph_loss_cos)
    hist["acc"].append(eph_cos_acc)
    hist["base_loss"].append(eph_base_loss)

    scheduler.step()
    flush_memory()
    cur_lr = scheduler.get_last_lr()[0]
    cur_lr = round(cur_lr, 5)

    # https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling
    LR_RATE*=0.97
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                                  cycle_momentum = False,
                                                  base_lr=LR_RATE/10,
                                                  max_lr=LR_RATE,
                                                  step_size_up=5,
                                                  mode="triangular2")

    try: print("diff_loss_weight", diff_loss_weight)
    except: pass
    print(f'\rEp {epoch} all_loss {hist["loss"][-1]:.5f} | acc {hist["acc"][-1]:.5f} | mse_loss {hist["loss_mse"][-1]:.5f} | cos_loss {hist["loss_cos"][-1]:.5f} | lr {cur_lr:.5f}\n')# {text}\n')

Пробегаемся по всем эпохам:   0%|          | 0/3 [00:00<?, ?EPOHS/s]

diff_loss_weight 0
Ep 0 all_loss 0.02125 | acc 0.39280 | mse_loss 0.01113 | cos_loss 0.01012 | lr 0.00027

diff_loss_weight 0
Ep 1 all_loss 0.02891 | acc 0.38917 | mse_loss 0.01481 | cos_loss 0.01410 | lr 0.00026

diff_loss_weight 0.022977941176470562
Ep 2 all_loss 0.02996 | acc 0.38574 | mse_loss 0.01578 | cos_loss 0.01418 | lr 0.00026

